In [1]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
entities = ["Alice", "Bob", "Charlie", "Dana"]
entity_to_id = {name: i for i, name in enumerate(entities)}

parent_facts = [
    ("Alice", "Bob"),
    ("Bob", "Charlie"),
    ("Charlie", "Dana"),
]

def parent_matrix(entities, parent_facts):
    n = len(entities)
    P = np.zeros((n, n), dtype=int)  # P[x,y] = 1 if Parent(x,y)
    for p, c in parent_facts:
        P[entity_to_id[p], entity_to_id[c]] = 1
    return P

P = parent_matrix(entities, parent_facts)
P


array([[0, 1, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 0]])

In [3]:
docs = [
    "Alice is the parent of Bob.",
    "Bob is the parent of Charlie.",
    "Charlie is the parent of Dana.",
]

vectorizer = TfidfVectorizer()
doc_vectors = vectorizer.fit_transform(docs)

def retrieve_docs(query, k=2):
    q_vec = vectorizer.transform([query])
    sims = cosine_similarity(q_vec, doc_vectors)[0]
    topk_idx = sims.argsort()[::-1][:k]
    return [(i, sims[i], docs[i]) for i in topk_idx]

query = "Is Alice a grandparent of Charlie?"
results = retrieve_docs(query, k=3)
for rank, (idx, score, text) in enumerate(results, start=1):
    print(f"Rank {rank} | score={score:.3f} | doc={text}")


Rank 1 | score=0.653 | doc=Alice is the parent of Bob.
Rank 2 | score=0.529 | doc=Bob is the parent of Charlie.
Rank 3 | score=0.490 | doc=Charlie is the parent of Dana.


In [4]:
def step(x):
    """Heaviside step: 1 if x > 0 else 0, elementwise."""
    return (x > 0).astype(int)

# P is Parent[x,y]

# Grandparent[x,z] = H( P[x,y] ⋅ P[y,z] )
G = step(np.einsum("xy,yz->xz", P, P))

print("Parent matrix P:")
print(P)
print("\nGrandparent matrix G = step(P @ P):")
print(G)


Parent matrix P:
[[0 1 0 0]
 [0 0 1 0]
 [0 0 0 1]
 [0 0 0 0]]

Grandparent matrix G = step(P @ P):
[[0 0 1 0]
 [0 0 0 1]
 [0 0 0 0]
 [0 0 0 0]]


In [5]:
def is_parent(x, y):
    return bool(P[entity_to_id[x], entity_to_id[y]])

def is_grandparent(x, y):
    return bool(G[entity_to_id[x], entity_to_id[y]])

def explain_grandparent(x, y):
    """Return one chain x -> m -> y if it exists."""
    i = entity_to_id[x]
    k = entity_to_id[y]
    if not G[i, k]:
        return None
    # find some middle node m: Parent(x,m) and Parent(m,y)
    for m in entities:
        j = entity_to_id[m]
        if P[i, j] and P[j, k]:
            return [x, m, y]
    return None

q_x, q_y = "Alice", "Charlie"
print("Parent facts:")
for p, c in parent_facts:
    print(f"Parent({p}, {c})")

print("\nQuery: Is Alice a grandparent of Charlie?")
print("Tensor-logic answer:", is_grandparent(q_x, q_y))

chain = explain_grandparent(q_x, q_y)
if chain:
    print("Explanation chain:", " -> ".join(chain))


Parent facts:
Parent(Alice, Bob)
Parent(Bob, Charlie)
Parent(Charlie, Dana)

Query: Is Alice a grandparent of Charlie?
Tensor-logic answer: True
Explanation chain: Alice -> Bob -> Charlie


In [6]:
def rag_like_answer(query, k=3):
    print("=== RAG-style retrieval ===")
    print("Query:", query)
    results = retrieve_docs(query, k=k)
    for rank, (idx, score, text) in enumerate(results, start=1):
        print(f"Rank {rank}: score={score:.3f} | {text}")
    print("Note: no explicit 'grandparent' fact; an LLM would need to reason over these.")

def tensor_logic_answer_grandparent(x, y):
    print("\n=== Tensor-Logic-style reasoning ===")
    print(f"Query: Is {x} a grandparent of {y}?")
    ans = is_grandparent(x, y)
    print("Answer:", ans)
    if ans:
        chain = explain_grandparent(x, y)
        if chain:
            print("Reason:", " -> ".join(chain))

rag_like_answer("Is Alice a grandparent of Charlie?")
tensor_logic_answer_grandparent("Alice", "Charlie")


=== RAG-style retrieval ===
Query: Is Alice a grandparent of Charlie?
Rank 1: score=0.653 | Alice is the parent of Bob.
Rank 2: score=0.529 | Bob is the parent of Charlie.
Rank 3: score=0.490 | Charlie is the parent of Dana.
Note: no explicit 'grandparent' fact; an LLM would need to reason over these.

=== Tensor-Logic-style reasoning ===
Query: Is Alice a grandparent of Charlie?
Answer: True
Reason: Alice -> Bob -> Charlie
